In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import linprog
%matplotlib inline

In [ ]:
def print_discrete_prob_distribution(p, color="blue"):
    plt.bar(range(l), p, 1, color=color, alpha=1)
    plt.ylim(0, 0.5)
    plt.show()

In [ ]:
l = 10

mu = np.array([0, 0, 0, 0, 4, 5, 8, 10, 13, 10])
nu = np.array([14, 15, 16, 10, 4, 1, 0, 0, 0, 0])
mu = mu / np.sum(mu)
nu = nu / np.sum(nu)

### Visualize probability distributions $\mu$ and $\nu$

In [ ]:
print("mu:")
print_discrete_prob_distribution(mu, color="blue")

print("nu:")
print_discrete_prob_distribution(nu, color="green")

### Euclidean barycenter between $\mu$ and $\nu$

In [ ]:
ts = [0, 0.25, 0.5, 0.75, 1]

for t in ts:
    print("barycenter: t=" + str(t))
    print_discrete_prob_distribution(t * mu + (1-t) * nu, color="green")

### Compute Wasserstein barycenter between $\mu$ and $\nu$

In [ ]:
C = [[0 for _ in range(l)] for _ in range(l)]

for i in range(l):
    for j in range(l):
        C[i][j] = abs(range(l)[i] - range(l)[j])**2

In [ ]:
ts = [0, 0.25, 0.5, 0.75, 1]

for t in ts:
    # TODO:
    # Construct matrix c as: [[  t*c  ]
    #                         [(1-t)*c]
    #                         [   0   ]]
    c = np.array(C).reshape((l**2))

    # TODO: 
    # Construct matrices of ones, A_r and A_t, which when multiplied by P reshaped to lxl vector 
    # gives us the equality contraints. Where row i of A_r equals sum of entries of P_i 
    # and row i of A_t equals sum of entries of row i of (P^T). 
    A_r = np.zeros((l, l, l))
    A_t = np.zeros((l, l, l))

    # TODO:
    # Construct matrix A of form: [[A_t, 0, 0]
    #                              [0, A_t, 0]
    #                              [A_r, 0,-I]
    #                              [0, A_r,-I]]
    A_eye = np.eye(l)
    A_zero_1 = np.zeros((A_eye.shape))
    A_zero_2 = np.zeros((l, l**2))
    A = np.zeros((l*4, 2*l**2 + 1))

    # Construct vector b as: [mu, nu, 0, 0].T
    b_zero = np.zeros(nu.shape)
    b = np.concatenate((mu, nu, b_zero, b_zero), axis=0)

    # TODO:
    # Solve LP with objective c^Tx, constraints Ax = b
    x = np.zeros((2*l**2 + l,))
  
    # Our resulting x = [P_1, P_2, a].T
    P_1 = x[:l**2]
    P_2 = x[l**2:2*l**2]
    a = x[2*l**2:]
    
    print("barycenter: t=" + str(t))
    
    # Uncomment this line to visualize barycenter results.
    # print_discrete_prob_distribution(a, color="green")